## Evaluation of clustering (shape-based)

- Use labeled data collected from anomaly_patterns_clustering_shape.ipynb.

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import jsonlines
import glob
from collections import defaultdict

In [3]:
for f in glob.glob("../samples/clustering_anomaly_patterns/*.jsonl"):
    with jsonlines.open(f) as reader:
        aggr = defaultdict(int)
        aggr2 = defaultdict(int)
        for obj in reader:
            aggr[obj["anomaly_pattern"]] += 1
            aggr2[obj["anomaly_position"]] += 1
    if len(aggr) < 1:
        continue
    display(f, aggr, aggr2)

'../samples/clustering_anomaly_patterns/clustering_anomaly_patterns_20221030-162851.jsonl'

defaultdict(int,
            {'Level shift down': 982,
             'Single spike': 458,
             'Level shift up': 550,
             'Multiple spikes': 321,
             'Single dip': 376,
             'Fluctuations': 86,
             'Other normal': 364,
             'Steady increase': 711,
             'Transient level shift up': 51,
             'Transient level shift down': 17,
             'White noise': 112,
             'Steady decrease': 52,
             'Sudden increase': 67,
             'Multiple dips': 12,
             'Sudden decrease': 49})

defaultdict(int,
            {'anomaly_during_fault': 3045,
             'anomaly_outside_fault': 695,
             'no_anomaly': 468})

'../samples/clustering_anomaly_patterns/clustering_anomaly_patterns_20221028-172414.jsonl'

defaultdict(int,
            {'Level shift up': 34,
             'Single dip': 49,
             'Single spike': 65,
             'Other normal': 12,
             'Multiple spikes': 34,
             'Steady increase': 8,
             'Transient level shift down': 4,
             'Transient level shift up': 10,
             'Level shift down': 10,
             'White noise': 8,
             'Multiple dips': 2,
             'Sudden increase': 2,
             'Fluctuations': 3,
             'Steady decrease': 3})

defaultdict(int,
            {'anomaly_during_fault': 149,
             'anomaly_outside_fault': 75,
             'no_anomaly': 20})

In [4]:
import numpy as np
import pandas as pd
import random
import scipy.interpolate
import scipy.stats

In [5]:
import sys
sys.path.append('../')

from tsdr import tsdr

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [6]:
fpath = "../samples/clustering_anomaly_patterns/clustering_anomaly_patterns_20221030-162851.jsonl"

samples: dict = {}
time_series_by_case: dict[tuple[str, str], list[tuple[str, np.ndarray]]] = defaultdict(list)
with jsonlines.open(fpath) as reader:
    for obj in reader:
        time_series_by_case[(obj["chaos_type"], obj["chaos_comp"])].append((obj["metric"], np.array(obj["time_series"])))
        
        key = (obj["chaos_type"], obj["chaos_comp"], obj["metric"])
        samples[key] = {"series": np.array(obj["time_series"], dtype=np.float64)}
        apos, apattern = obj["anomaly_position"], obj["anomaly_pattern"]
        if apos == "no_anomaly" or apattern in ["White noise", "Other normal"]:
            samples[key].update({
                "anomaly_type": "type0",
                "anomaly_pattern": "normal",
                "anomaly_position": apos,
            })
        else:
            match apattern:
                # Type 1
                case "Level shift down" | "Level shift up" | "Steady decrease" | "Steady increase" | "Sudden decrease" | "Sudden increase":
                    samples[key].update({
                        "anomaly_type": "type1",
                        "anomaly_pattern": apattern,
                        "anomaly_position": apos,
                    })
                # Type 2
                case "Fluctuations" | "Multiple dips" | "Multiple spikes" | "Single dip" | "Single spike" | "Transient level shift down" | "Transient level shift up":
                    samples[key].update({
                        "anomaly_type": "type2",
                        "anomaly_pattern": apattern,
                        "anomaly_position": apos,
                    })

In [19]:
from meltria.priorknowledge import priorknowledge
from joblib import Parallel, delayed


pk = priorknowledge.new_knowledge(
    target_app="train-ticket",
    target_metric_types={
        "containers": True,
        "services": True,
        "middlewalres": True,
        "nodes": False,
    },
    mappings={"nodes-containers": {}},
)


def _clustering(pk, time_series):
    metric_name_to_values = {metric: scipy.stats.zscore(values) for metric, values in time_series}
    _, clustering_info = tsdr.Tsdr("residual_integral", **{
        "step2_clustering_method_name": "dbscan",
        "step2_dbscan_min_pts": 2,
        "step2_dbscan_dist_type": "sbd",  # 'pearsonr' or 'sbd'
        "step2_dbscan_algorithm": "hdbscan",  # 'dbscan' or 'hdbscan'
        "step2_clustering_series_type": "raw",  # 'raw', 'anomaly_score' or 'binary_anomaly_score'
        "step2_clustering_choice_method": 'medoid',  # 'medoid' or 'maxsum'
    }).reduce_multivariate_series(pd.DataFrame(metric_name_to_values), pk, n_workers=1)
    return clustering_info

clustering_infos = Parallel(n_jobs=-1)(delayed(_clustering)(pk, ts) for ts in time_series_by_case.values())

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

In [20]:
from itertools import combinations

def _create_df_from_clustering_infos(_clustering_infos: list) -> pd.DataFrame:
    eval_stat: list[tuple[str, str, int, str, int, int, int]] = []
    for i, ((chaos_type, chaos_comp), time_series) in enumerate(time_series_by_case.items()):
        clustering_info = _clustering_infos[i]
        for i, (representative_metric, sub_metrics) in enumerate(clustering_info.items(), start=1):
            positives, negatives = 0, 0
            for u, v in combinations([representative_metric] + sub_metrics, 2):
                u_atype: str = samples[chaos_type, chaos_comp, u]["anomaly_type"]
                v_atype: str = samples[chaos_type, chaos_comp, v]["anomaly_type"]
                if u_atype == v_atype:
                    positives += 1
                else:
                    negatives += 1

            eval_stat.append((chaos_type, chaos_comp, i, representative_metric, positives, negatives, len(sub_metrics)+1))

    return pd.DataFrame(eval_stat, columns=["chaos_type", "chaos_comp", "cluster_no", "rep", "positives", "negatives", "total_metrics"]).reset_index().set_index(["chaos_type", "chaos_comp", "cluster_no"])

eval_df = _create_df_from_clustering_infos(clustering_infos)
pd.options.display.max_rows = None
eval_df

index  \
chaos_type       chaos_comp          cluster_no          
pod-memory-hog   ts-preserve-service 1               0   
                                     2               1   
                                     3               2   
                                     4               3   
                                     5               4   
                                     6               5   
                                     7               6   
                                     8               7   
                                     9               8   
                                     10              9   
                                     11             10   
                                     12             11   
                                     13             12   
                                     14             13   
                                     15             14   
                                     16             15   
                                     17             16   
                                     18             17   
                                     19             18   
                                     20             19   
                                     21             20   
                 ts-order-service    1              21   
                                     2              22   
                                     3              23   
                                     4              24   
                                     5              25   
                                     6              26   
                                     7              27   
                                     8              28   
                                     9              29   
                                     10             30   
                                     11             31   
                                     12             32   
                                     13             33   
                                     14             34   
                                     15             35   
                                     16             36   
                                     17             37   
                                     18             38   
                                     19             39   
                                     20             40   
                                     21             41   
                                     22             42   
                                     23             43   
                                     24             44   
                                     25             45   
                                     26             46   
                                     27             47   
                                     28             48   
                                     29             49   
                                     30             50   
                                     31             51   
                                     32             52   
                                     33             53   
                                     34             54   
                                     35             55   
                                     36             56   
                                     37             57   
                                     38             58   
                                     39             59   
                                     40             60   
                                     41             61   
                                     42             62   
                                     43             63   
                                     44             64   
                                     45             65   
                                     46             66   
                                     47        

In [15]:
eval_df_sum = eval_df.groupby(["chaos_type", "chaos_comp"]).apply(lambda x: x["positives"].sum() / (x["positives"].sum() + x["negatives"].sum()))
eval_df_sum

chaos_type        chaos_comp         
pod-cpu-hog       ts-food-service        0.503658
                  ts-order-mongo         0.504456
                  ts-station-service     0.625195
                  ts-train-service       0.483926
                  ts-travel-service      0.709742
pod-memory-hog    ts-consign-mongo       0.548535
                  ts-order-service       0.790629
                  ts-preserve-service    0.714012
                  ts-station-service     0.504543
                  ts-train-mongo         0.570673
pod-network-loss  ts-auth-mongo          0.774283
                  ts-basic-service       0.637363
                  ts-price-mongo         0.876582
                  ts-travel-mongo        0.825894
                  ts-travel2-service     0.564732
dtype: float64

In [16]:
eval_df_sum.groupby(["chaos_type"]).mean()

chaos_type
pod-cpu-hog         0.565395
pod-memory-hog      0.625678
pod-network-loss    0.735771
dtype: float64

In [17]:
eval_df_sum.mean()

0.6422814480435749

### Comparison with the existing method (FluxRank)

In [21]:
def _fluxrank_clustering(pk, time_series):
    metric_name_to_values = {metric: values for metric, values in time_series}
    _, clustering_info = tsdr.Tsdr("residual_integral", **{
        "step2_clustering_method_name": "dbscan",
        "step2_dbscan_min_pts": 2,
        "step2_dbscan_dist_type": "pearsonr",  # 'pearsonr' or 'sbd'
        "step2_dbscan_algorithm": "dbscan",  # 'dbscan' or 'hdbscan'
        "step2_clustering_series_type": "raw",  # 'raw', 'anomaly_score' or 'binary_anomaly_score'
        "step2_clustering_choice_method": 'medoid',  # 'medoid' or 'maxsum'
    }).reduce_multivariate_series(pd.DataFrame(metric_name_to_values), pk, n_workers=1)
    return clustering_info

fluxrank_clustering_infos = Parallel(n_jobs=-1)(delayed(_fluxrank_clustering)(pk, ts) for ts in time_series_by_case.values())

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.10/site-packages/sklearn/neighbors/_base.py:199: EfficiencyWarning: Precomputed sparse input was not sorted by data.
  warnings.warn(


INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

In [29]:
fluxrank_eval_df = _create_df_from_clustering_infos(fluxrank_clustering_infos)
pd.options.display.max_rows = None
fluxrank_eval_df

index  \
chaos_type       chaos_comp          cluster_no          
pod-memory-hog   ts-preserve-service 1               0   
                                     2               1   
                                     3               2   
                 ts-order-service    1               3   
                                     2               4   
                                     3               5   
                                     4               6   
                 ts-consign-mongo    1               7   
                                     2               8   
                                     3               9   
                                     4              10   
                                     5              11   
                 ts-station-service  1              12   
                                     2              13   
                                     3              14   
                                     4              15   
                                     5              16   
                 ts-train-mongo      1              17   
                                     2              18   
                                     3              19   
                                     4              20   
pod-cpu-hog      ts-train-service    1              21   
                                     2              22   
                                     3              23   
                                     4              24   
                                     5              25   
                                     6              26   
                 ts-food-service     1              27   
                                     2              28   
                                     3              29   
                                     4              30   
                                     5              31   
                 ts-travel-service   1              32   
                                     2              33   
                                     3              34   
                                     4              35   
                                     5              36   
                 ts-station-service  1              37   
                                     2              38   
                                     3              39   
                                     4              40   
                 ts-order-mongo      1              41   
                                     2              42   
                                     3              43   
                                     4              44   
pod-network-loss ts-travel-mongo     1              45   
                                     2              46   
                                     3              47   
                                     4              48   
                 ts-travel2-service  1              49   
                                     2              50   
                                     3              51   
                                     4              52   
                 ts-basic-service    1              53   
                                     2              54   
                 ts-auth-mongo       1              55   
                                     2              56   
                                     3              57   
                                     4              58   
                 ts-price-mongo      1              59   
                                     2              60   
                                     3              61   
                                     4              62   

                                                                                               rep  \
chaos_type       chaos_comp          cluster_no                                                      
pod-memory-hog   ts-preserve-service 1                                s-ts

In [26]:
fluxrank_eval_df_sum = fluxrank_eval_df.groupby(["chaos_type", "chaos_comp"]).apply(lambda x: x["positives"].sum() / (x["positives"].sum() + x["negatives"].sum()))
fluxrank_eval_df_sum

chaos_type        chaos_comp         
pod-cpu-hog       ts-food-service        0.503658
                  ts-order-mongo         0.504456
                  ts-station-service     0.625195
                  ts-train-service       0.483926
                  ts-travel-service      0.709742
pod-memory-hog    ts-consign-mongo       0.548535
                  ts-order-service       0.790629
                  ts-preserve-service    0.714012
                  ts-station-service     0.504543
                  ts-train-mongo         0.570673
pod-network-loss  ts-auth-mongo          0.774283
                  ts-basic-service       0.637363
                  ts-price-mongo         0.876582
                  ts-travel-mongo        0.825894
                  ts-travel2-service     0.564732
dtype: float64

In [27]:
fluxrank_eval_df_sum.groupby(["chaos_type"]).mean()

chaos_type
pod-cpu-hog         0.565395
pod-memory-hog      0.625678
pod-network-loss    0.735771
dtype: float64

In [28]:
fluxrank_eval_df_sum.mean()

0.6422814480435749